In [ ]:
!python -V

Python 3.9.7


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
%cd gdrive/My Drive/

[Errno 2] No such file or directory: 'gdrive/My Drive/'
/content


In [ ]:
! git clone https://github.com/DataTalksClub/mlops-zoomcamp

Cloning into 'mlops-zoomcamp'...
remote: Enumerating objects: 1825, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 1825 (delta 47), reused 99 (delta 42), pack-reused 1695
Receiving objects: 100% (1825/1825), 3.93 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (931/931), done.


In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet

--2023-05-23 17:46:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.51, 65.8.245.50, 65.8.245.178, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36.37M  69.5MB/s    in 0.5s    

2023-05-23 17:46:42 (69.5 MB/s) - ‘yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]



In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

--2023-05-23 17:47:31--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.171, 65.8.245.178, 65.8.245.50, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45616512 (44M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-02.parquet’

yellow_tripdata_202 100%[===================>]  43.50M  68.4MB/s    in 0.6s    

2023-05-23 17:47:32 (68.4 MB/s) - ‘yellow_tripdata_2022-02.parquet’ saved [45616512/45616512]



In [ ]:
import pandas as pd

In [ ]:
import pickle

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet('yellow_tripdata_2022-01.parquet')
df.columns.values
initial_count = len(df)

In [ ]:
df = pd.read_parquet('yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('yellow_tripdata_2022-02.parquet')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

# df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [ ]:
len(list(df.columns.values))

20

In [ ]:
df['duration'].std()

46.44530513776802

In [ ]:
df_val = pd.read_parquet('yellow_tripdata_2022-02.parquet')

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_val[categorical] = df_val[categorical].astype(str)

In [ ]:
df_val['duration'].std()

9.6430980172662

In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df

remaining_count = len(df)

percentage_remaining = (remaining_count / initial_count) * 100
percentage_remaining

98.27547930522405

In [ ]:
df[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.001496179427095

In [ ]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_val = dv.transform(val_dicts)
y_val = df_val['duration'].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.795498607717361

In [ ]:
X_train.shape

(2421440, 516)